In [30]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

In [2]:
df = pd.read_csv('clean_tweets.csv')

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df['sentiment'] = df['sentiment'].map({0:0,4:1})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
sentiment    1600000 non-null int64
text         1596714 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [5]:
df.dropna(inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1596714 entries, 0 to 1599999
Data columns (total 2 columns):
sentiment    1596714 non-null int64
text         1596714 non-null object
dtypes: int64(1), object(1)
memory usage: 36.5+ MB


## Train/ Dev/ Test Split

We will split the data into three sections: train, development and test. Our chosen ratio is 98/1/1 i.e. 98% for the training set, 1% for the development set and 1% for the testing set.
* Train set: The dataset used for learning
* Development Set: A validation/development dataset is a sample of data held back from training your model that is used to give an estimate of model skill while tuning model’s hyperparameters.
* Test Set: The dataset used to assess the performance of a model.


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x = df['text'] #define all other columns except the target variable
y = df['sentiment'] #define the target variable

x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size = 0.02, random_state = 42)

x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, 
                                                              test_size = 0.5, random_state = 42)

In [9]:
print("Training set has {0} entries, where {1:.2f} are positive and {2:.2f} are negative".
      format(len(x_train),len(x_train[y_train==1])/len(x_train)*100, len(x_train[y_train==0])/len(x_train)*100))
print("Validation set has {0} entries, where {1:.2f} are positive and {2:.2f} are negative".
      format(len(x_validation),len(x_validation[y_validation==1])/len(x_validation)*100, 
             len(x_validation[y_validation==0])/len(x_validation)*100))
print("Testing set has {0} entries, where {1:.2f} are positive and {2:.2f} are negative".
      format(len(x_test),len(x_test[y_test==1])/len(x_test)*100, 
             len(x_test[y_test==0])/len(x_test)*100))

Training set has 1564779 entries, where 49.99 are positive and 50.01 are negative
Validation set has 15967 entries, where 49.82 are positive and 50.18 are negative
Testing set has 15968 entries, where 50.33 are positive and 49.67 are negative


## Baseline

### TextBlob

We will use the sentiment analysis function of TextBlob as baseline for our project. It will provide us a point of reference to compare our future models.

In [10]:
from textblob import TextBlob

In [11]:
tbresult = []
for i in x_validation:
    tbresult.append(TextBlob(i).sentiment.polarity)

In [12]:
tbpred = []
for i in tbresult:
    if i<0:
        tbpred.append(0)
    else:
        tbpred.append(1)    

In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [19]:
conmat = np.array(confusion_matrix(y_validation, tbpred, labels=[1,0]))
confusion = pd.DataFrame(conmat, index=['positive', 'negative'], columns=['predicted_positive', 'predicted_negative'])
print("Accuracy score: {0:.2f} %".format(accuracy_score(y_validation, tbpred)*100))

Accuracy score: 61.41 %


In [20]:
print("Confusion Matrix")
print(confusion)

Confusion Matrix
          predicted_positive  predicted_negative
positive                7136                 818
negative                5344                2669


In [21]:
print("Classification Report")
print(classification_report(y_validation, tbpred))

Classification Report
              precision    recall  f1-score   support

           0       0.77      0.33      0.46      8013
           1       0.57      0.90      0.70      7954

   micro avg       0.61      0.61      0.61     15967
   macro avg       0.67      0.62      0.58     15967
weighted avg       0.67      0.61      0.58     15967



Hence, TextBlob sentiment analysis yielded 61.41% accuracy on the validation set.

## Feature Extraction

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time

Pipeline class allows sticking multiple processes into a single scikit-learn estimator.

In [25]:
null_accuracy = 0
def accuracy_summary(pipeline, x_train, y_train, x_test, y_test):
    if len(x_test[y_test==0])/len(x_test)>0.5:
        null_accuracy = len(x_test[y_test==0])/len(x_test)
    else:
        null_accuracy = 1 - len(x_test[y_test==0])/len(x_test)
    t0 = time()
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    train_test_time = time() - t0
    accuracy = accuracy_score(y_test, y_pred)
    print("Null accuracy: {0:.2f}%".format(null_accuracy*100))
    print("Accuracy: {0:.2f}%".format(accuracy*100))
    if accuracy>null_accuracy:
        print("Model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100))
    elif accuracy==null_accuracy:
        print("Model has the same accuracy as null accuracy")
    else:
        print("Model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100))
    print("Train and test time: {0:.2f}s".format(train_test_time))
    print("-"*50)
    return accuracy, train_test_time

In [26]:
cvec = CountVectorizer()
lr = LogisticRegression()
n_features = np.arange(10000, 100001, 10000)

In [28]:
def nfeature_accuracy_checker(vectorizer = cvec, n_features = n_features, stop_words = None, 
                              ngram_range = (1,1), classifier = lr):
    result = []
    print(classifier, "\n")
    for n in n_features:
        vectorizer.set_params(stop_words = stop_words, max_features = n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([('vectorizer', vectorizer), ('classifier', classifier)])
        print("Validation result for {0} features".format(n))
        nfeature_accuracy, ttime = accuracy_summary(checker_pipeline, x_train, y_train, x_validation, y_validation)
        result.append((n, nfeature_accuracy, ttime))
    return result

# Unigram

## Without stop words

In [32]:
%%time
print("RESULT FOR UNIGRAM WITHOUT STOP WORDS\n")
feature_result_unigram = nfeature_accuracy_checker(stop_words='english')

RESULT FOR UNIGRAM WITHOUT STOP WORDS

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False) 

Validation result for 10000 features


C:\Users\dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Null accuracy: 50.18%
Accuracy: 77.27%
Model is 27.08% more accurate than null accuracy
Train and test time: 101.11s
--------------------------------------------------
Validation result for 20000 features
Null accuracy: 50.18%
Accuracy: 77.52%
Model is 27.33% more accurate than null accuracy
Train and test time: 118.81s
--------------------------------------------------
Validation result for 30000 features
Null accuracy: 50.18%
Accuracy: 77.54%
Model is 27.36% more accurate than null accuracy
Train and test time: 138.48s
--------------------------------------------------
Validation result for 40000 features
Null accuracy: 50.18%
Accuracy: 77.52%
Model is 27.34% more accurate than null accuracy
Train and test time: 163.33s
--------------------------------------------------
Validation result for 50000 features
Null accuracy: 50.18%
Accuracy: 77.50%
Model is 27.32% more accurate than null accuracy
Train and test time: 144.55s
--------------------------------------------------
Validation r

## With Stop Words

In [33]:
%%time
print("RESULT FOR UNIGRAM WITH STOP WORDS\n")
feature_result_unigram_stop = nfeature_accuracy_checker()

RESULT FOR UNIGRAM WITH STOP WORDS

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False) 

Validation result for 10000 features
Null accuracy: 50.18%
Accuracy: 79.73%
Model is 29.55% more accurate than null accuracy
Train and test time: 138.26s
--------------------------------------------------
Validation result for 20000 features
Null accuracy: 50.18%
Accuracy: 79.90%
Model is 29.71% more accurate than null accuracy
Train and test time: 181.06s
--------------------------------------------------
Validation result for 30000 features
Null accuracy: 50.18%
Accuracy: 79.98%
Model is 29.80% more accurate than null accuracy
Train and test time: 190.65s
--------------------------------------------------
Validation result for 40000 features
Null accuracy: 50.18%
Accuracy: 80.02%
Model i